In [1]:
import pandas as pd
import sys
sys.path.append('./utils')
from utils import *
from rank import *
from process import * 
from tqdm.notebook import tqdm

In [2]:
# データ保存先
data_folda = '../data/'

In [3]:
# make_timeseries_data.ipynb で作成したデータを読み込む
pseudo_time_series=pd.read_pickle(data_folda+'skeleton_train.pkl')
# base_date が 2023年1月1日以降のデータを抽出 (closed_date がそれ以降の閉店店舗しかクロールしていないため)
# 3ヵ月前の閉店データを使うため
pseudo_time_series=pseudo_time_series[pseudo_time_series['base_date']>='2022-09-01']

add_list=['name',"prefecture_name",'head_branch','opened_on', 'created_at','genre_first_name', 
              'genre_second_name','genre_third_name','northlatitude', 'eastlongitude',
              "net_reservation_flg",'price_range_lunch_owner','price_range_dinner_owner', 
              'price_range_lunch_user','price_range_dinner_user',
              # 20240312追加
              'all_photo_count','photo_food_count','photo_drink_count',
              'photo_interior_count','photo_exterior_count','official_info_flg',
              'max_visit_month'
              ]


tabelog = pd.read_csv(data_folda+"restaurant_data.csv")
    
pseudo_time_series=pd.merge(pseudo_time_series, tabelog[["id"]+add_list], left_on="original_id",right_on='id', how='left')

* masterからデータを追加

In [5]:
### master data から加えるデータを定義
add_list=['name',"prefecture_name",'head_branch','opened_on', 'created_at','genre_first_name', 
              'genre_second_name','genre_third_name','northlatitude', 'eastlongitude',
              "net_reservation_flg",'price_range_lunch_owner','price_range_dinner_owner', 
              'price_range_lunch_user','price_range_dinner_user',
              # 20240312追加
              'all_photo_count','photo_food_count','photo_drink_count',
              'photo_interior_count','photo_exterior_count','official_info_flg',
              'max_visit_month'
              ]

# make_timeseries_data.ipynb で作成したデータを読み込む
pseudo_time_series=pd.read_pickle(data_folda+'skeleton_train.pkl')
# base_date が 2023年1月1日以降のデータを抽出 (closed_date がそれ以降の閉店店舗しかクロールしていないため)
pseudo_time_series=pseudo_time_series[pseudo_time_series['base_date']>='2022-12-31']
pseudo_time_series = add_master_data(pseudo_time_series, data_folda, add_list, "restaurant_data.csv")
pseudo_time_series.to_pickle(data_folda+'train_with_master.pkl')


TypeError: can only concatenate list (not "str") to list

In [6]:
pseudo_time_series_test3=pd.read_pickle(data_folda+'skeleton_test_3month.pkl')
pseudo_time_series_test3 = add_master_data(pseudo_time_series_test3, data_folda, add_list, "restaurant_data.csv")
pseudo_time_series_test3.to_pickle(data_folda+'skeleton_test_3month_master.pkl')

pseudo_time_series_test6=pd.read_pickle(data_folda+'skeleton_test_6month.pkl')
pseudo_time_series_test6 = add_master_data(pseudo_time_series_test6, data_folda, add_list, "restaurant_data.csv")
pseudo_time_series_test6.to_pickle(data_folda+'skeleton_test_6month_master.pkl')

In [12]:
def calculate_close_ratio(pseudo_time_series, data_folda):
    #将来ジャンルが変わってモデルの構成が変わる可能性があるため、ジャンルをリストで保持して読み込む
    #genre_list = make_genre_list(pseudo_time_series)
    genre_list = pd.read_pickle(data_folda+'genre_list.pkl')
    
    close_ratios=pd.DataFrame()
    eval_period=pd.date_range('2022-09-01', '2023-12-01', freq='MS')
    #eval_period から一日前の日付を取得
    eval_period=eval_period - pd.Timedelta(days=1)

    # tqdm で進行状況を表示
    for d in tqdm(range(3,len(eval_period))):
        for g in tqdm(genre_list,leave=True):
            # base_data の3ヶ月前のデータで、eval_period がTrueのidを取得
            num_3months_ago = pseudo_time_series[(pseudo_time_series['base_date'] == eval_period[d-3])
                                                 &(pseudo_time_series['genre_first_name'] == g)
                                                 &(pseudo_time_series['eval_period'] == True)]
            
            id_3months_ago = num_3months_ago['id'].values
            num_this_month = pseudo_time_series[(pseudo_time_series['base_date'] == eval_period[d])
                                                &(pseudo_time_series['id'].isin(id_3months_ago)
                                                &(pseudo_time_series['eval_period'] == True))]
            
            try:
                ratio=len(num_this_month)/len(num_3months_ago)
            except ZeroDivisionError:
                ratio=np.nan
                
            close_ratios.append([eval_period[d],g,ratio])
    
    close_ratios.rename(columns={0:"base_date",1:"genre_first_name",2:"close_ratio"},inplace=True)

    return close_ratios

In [13]:
close_ratio=calculate_close_ratio(pseudo_time_series,data_folda)

  0%|          | 0/13 [00:00<?, ?it/s]

/tmp/ipykernel_18270/3093606114.py:29: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  close_ratios.append([eval_period[d],g,ratio])


In [7]:

pseudo_time_series=pd.merge(pseudo_time_series,close_ratio,on=["base_date","genre_first_name"],how="left")
pseudo_time_series.to_pickle(data_folda+'train_with_master.pkl')
pseudo_time_series_test3=pd.merge(pseudo_time_series_test3,close_ratio,on=["base_date","genre_first_name"],how="left")
pseudo_time_series_test3.to_pickle(data_folda+'skeleton_test_3month_master.pkl')
pseudo_time_series_test6=pd.merge(pseudo_time_series_test6,close_ratio,on=["base_date","genre_first_name"],how="left")
pseudo_time_series_test6.to_pickle(data_folda+'skeleton_test_6month_master.pkl')

del pseudo_time_series, pseudo_time_series_test3, pseudo_time_series_test6

  0%|          | 0/21 [00:00<?, ?it/s]

ZeroDivisionError: division by zero

* V_GOOGLE からデータを追加

In [ ]:
pseudo_time_series = pd.read_pickle(data_folda+'train_with_master.pkl')
add_list=["restaurant_id","overview.aggregateRating","overview.reviewCount","general_rank"]
pseudo_time_series = add_v_google(pseudo_time_series, data_folda, add_list)
pseudo_time_series.to_pickle(data_folda+'train_with_master_google.pkl')

pseudo_time_series_test = pd.read_pickle(data_folda+'skeleton_test_3month_master.pkl')
add_list=["restaurant_id","overview.aggregateRating","overview.reviewCount","general_rank"]
pseudo_time_series_test = add_v_google(pseudo_time_series_test, data_folda, add_list)
pseudo_time_series_test.to_pickle(data_folda+'skeleton_test_3month_master_google.pkl')

pseudo_time_series_test = pd.read_pickle(data_folda+'skeleton_test_6month_master.pkl')
add_list=["restaurant_id","overview.aggregateRating","overview.reviewCount","general_rank"]
pseudo_time_series_test = add_v_google(pseudo_time_series_test, data_folda, add_list)
pseudo_time_series_test.to_pickle(data_folda+'skeleton_test_6month_master_google.pkl')

del pseudo_time_series, pseudo_time_series_test

/home/ec2-user/dglab-ai-scoring_insitestream/pg/./utils/rank.py:68: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df["n_rank_count"] = df["telephone_rank"].isin(["N"]).astype(int) + df["address_rank"].isin(["N"]).astype(int) + df["name_rank"].isin(["N"]).astype(int)
/home/ec2-user/dglab-ai-scoring_insitestream/pg/./utils/rank.py:73: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df["denominator"] = np.select(condition, choice, default=-1)
/home/ec2-user/dglab-ai-scoring_insitestream/pg/./utils/rank.py:78: Se

* V_RETTY からデータを追加

In [ ]:
#追加する変数をリストで定義
add_list=["restaurant_id",'infos.isOfficial', 'infos.familiar.users.genge',
       'infos.familiar.users.count', 'infos.familiar.stars',
       'general_rank_RETTY', 'infos.reserveEntireStore',
       'infos.onlineReservation', 'infos.access.transferTime1','infos.access.transferTime2','infos.access.transferTime3',
       'infos.numberOfSeats.value', 'infos.counterSeats.value',
       'infos.privateRoom.value', 'infos.updateInfo.firstReviewDate',
       'infos.wantToGo', 'infos.updateInfo.lastReviewDate',
       'infos.went', 'infos.rateByWent', 'infos.excellent', 'infos.coupon',
       'infos.photoCount.Photograph', 'infos.photoCount.Cooking',
       'infos.photoCount.Interior', 'infos.photoCount.Exterior',
       'infos.photoCount.Menu', 'infos.reviewCount.Review',
       'infos.reviewCount.Lunch', 'infos.reviewCount.Dinner',
       'menus.course_list_count',
       'menus.dishes_list_count', 'menus.lunch_list_count',
       'menus.drink_list_count', 'menus.takeout_list_count']
 
pseudo_time_series = pd.read_pickle(data_folda+'train_with_master_google.pkl')         
pseudo_time_series = add_v_retty(pseudo_time_series, data_folda, add_list, master_name="V_RETTY",read_from_snowflake=False)
pseudo_time_series.to_pickle(data_folda+'train_with_master_google_retty.pkl')

pseudo_time_series_test = pd.read_pickle(data_folda+'skeleton_test_3month_master_google.pkl')
pseudo_time_series_test = add_v_retty(pseudo_time_series_test, data_folda, add_list, master_name="V_RETTY",read_from_snowflake=False)
pseudo_time_series_test.to_pickle(data_folda+'skeleton_test_3month_master_google_retty.pkl')

pseudo_time_series_test = pd.read_pickle(data_folda+'skeleton_test_6month_master_google.pkl')
pseudo_time_series_test = add_v_retty(pseudo_time_series_test, data_folda, add_list, master_name="V_RETTY",read_from_snowflake=False)
pseudo_time_series_test.to_pickle(data_folda+'skeleton_test_6month_master_google_retty.pkl')

del pseudo_time_series, pseudo_time_series_test

TypeError: add_v_retty() got an unexpected keyword argument 'read_from_snowflake'

* HOTPEPPER からデータを追加

In [ ]:
add_list=['restaurant_id','infos.aggregateRating','infos.ratingReview',
          'infos.satisfaction.percentage1','infos.satisfaction.percentage2','infos.satisfaction.percentage3',
          'infos.couponCount','infos.onlineReservation','infos.ownerRegistration','infos.ticket',
          'photos.allCount','photos.appearanceCount','photos.drinkCount','photos.foodCount','photos.otherCount',
          'photos.postCount','menus.courseListCount']


pseudo_time_series = pd.read_pickle(data_folda+'train_with_master_google_retty.pkl')
pseudo_time_series = add_v_hotpepper(pseudo_time_series, data_folda, add_list, master_name="V_HOTPEPPER",read_from_snowflake=False)
pseudo_time_series.to_pickle(data_folda+'train_with_master_google_retty_hotpepper.pkl')

pseudo_time_series_test = pd.read_pickle(data_folda+'skeleton_test_3month_master_google_retty.pkl')
pseudo_time_series_test = add_v_hotpepper(pseudo_time_series_test, data_folda, add_list, master_name="V_HOTPEPPER",read_from_snowflake=False) 
pseudo_time_series_test.to_pickle(data_folda+'skeleton_test_3month_master_google_retty_hotpepper.pkl')

pseudo_time_series_test = pd.read_pickle(data_folda+'skeleton_test_6month_master_google_retty.pkl')
pseudo_time_series_test = add_v_hotpepper(pseudo_time_series_test, data_folda, add_list, master_name="V_HOTPEPPER",read_from_snowflake=False) 
pseudo_time_series_test.to_pickle(data_folda+'skeleton_test_6month_master_google_retty_hotpepper.pkl')

del pseudo_time_series, pseudo_time_series_test

/home/ec2-user/dglab-ai-scoring_insitestream/pg/./utils/rank.py:68: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df["n_rank_count"] = df["telephone_rank"].isin(["N"]).astype(int) + df["address_rank"].isin(["N"]).astype(int) + df["name_rank"].isin(["N"]).astype(int)
/home/ec2-user/dglab-ai-scoring_insitestream/pg/./utils/rank.py:73: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df["denominator"] = np.select(condition, choice, default=-1)
/home/ec2-user/dglab-ai-scoring_insitestream/pg/./utils/rank.py:78: Se

* V_STRUCTURED からデータを取得

In [ ]:
add_list = '"restaurant_id","url","name_rank","address_rank","telephone_rank","aggregateRating.ratingCount","aggregateRating.ratingValue", "aggregateRating.reviewCount", \
    "aggregateRating.bestRating", "aggregateRating.worstRating", "price", \
    "priceRange", "servesCuisine","interactionCount.went", \
    "interactionCount.wanttogo","paymentAccepted", \
    "acceptsReservations", \
    "ReserveAction.result.name", "OrderAction.target.actionPlatform", \
    "OrderAction.target.inLanguage", "OrderAction.target.urlTemplate"'


pseudo_time_series = pd.read_pickle(data_folda+'train_with_master_google_retty_hotpepper.pkl')
pseudo_time_series = add_v_structured(pseudo_time_series, data_folda, add_list, master_name="V_STRUCTURED",read_from_snowflake=True)
pseudo_time_series.to_pickle(data_folda+'train_with_master_google_retty_hotpepper_structured.pkl')

del pseudo_time_series

pseudo_time_series_test = pd.read_pickle(data_folda+'skeleton_test_3month_master_google_retty_hotpepper.pkl')
pseudo_time_series_test = add_v_structured(pseudo_time_series_test, data_folda, add_list, master_name="V_STRUCTURED",read_from_snowflake=True) 
pseudo_time_series_test.to_pickle(data_folda+'skeleton_test_3month_master_google_retty_hotpepper_structured.pkl')

pseudo_time_series_test = pd.read_pickle(data_folda+'skeleton_test_6month_master_google_retty_hotpepper.pkl')
pseudo_time_series_test = add_v_structured(pseudo_time_series_test, data_folda, add_list, master_name="V_STRUCTURED",read_from_snowflake=True) 
pseudo_time_series_test.to_pickle(data_folda+'skeleton_test_6month_master_google_retty_hotpepper_structured.pkl')

del pseudo_time_series_test

KeyboardInterrupt: 

* V_TRIPADVISOR から特徴量を追加

In [ ]:
pd_sf.columns.values

array(['acquisition_id', 'url_id', 'url', 'crawl_stamp', 'restaurant_id',
       'tabelog_name', 'tabelog_branch_name', 'tabelog_address',
       'tabelog_telephone', 'tabelog_reservation_telephone', 'name_rank',
       'name_rank_system', 'name_ratio.ratio',
       'name_ratio.master_normalized_name', 'name_ratio.normalized_name',
       'name_ratio.include', 'name_ratio.diff', 'name_only_ratio.ratio',
       'name_only_ratio.master_normalized_name',
       'name_only_ratio.normalized_name', 'name_only_ratio.include',
       'name_only_ratio.diff', 'address_rank', 'address_rank_system',
       'postal.ratio', 'postal.master_value', 'postal.value',
       'postal.diff', 'region.ratio', 'region.master_value',
       'region.value', 'region.diff', 'street_address.ratio',
       'street_address.master_value', 'street_address.value',
       'street_address.diff', 'building_name.ratio',
       'building_name.master_value', 'building_name.value',
       'building_name.diff', 'telephone_rank'

In [ ]:
add_list=["restaurant_id",'infos.aggregateRating_TRIPADVISOR',"infos.reviewCount","infos.qa_count","infos.ratingDetails.Meal","infos.ratingDetails.Service","infos.ratingDetails.Price","infos.ratingDetails.Ambience"]

pseudo_time_series = pd.read_pickle(data_folda+'train_with_master_google_retty_hotpepper_structured.pkl')
pseudo_time_series = reduce_mem_usage(pseudo_time_series)
pseudo_time_series = add_v_tripadvisor(pseudo_time_series, data_folda, add_list, master_name="V_TRIPADVISOR",read_from_snowflake=False)
pseudo_time_series.to_pickle(data_folda+'train_with_master_google_retty_hotpepper_structured_tripadvisor.pkl')

del pseudo_time_series

pseudo_time_series_test = pd.read_pickle(data_folda+'skeleton_test_3month_master_google_retty_hotpepper_structured.pkl')
pseudo_time_series_test = reduce_mem_usage(pseudo_time_series_test)
pseudo_time_series_test = add_v_tripadvisor(pseudo_time_series_test, data_folda, add_list, master_name="V_TRIPADVISOR",read_from_snowflake=False) 
pseudo_time_series_test.to_pickle(data_folda+'skeleton_test_3month_master_google_retty_hotpepper_structured_tripadvisor.pkl')

pseudo_time_series_test = pd.read_pickle(data_folda+'skeleton_test_6month_master_google_retty_hotpepper_structured.pkl')
pseudo_time_series_test = reduce_mem_usage(pseudo_time_series_test)
pseudo_time_series_test = add_v_tripadvisor(pseudo_time_series_test, data_folda, add_list, master_name="V_TRIPADVISOR",read_from_snowflake=False) 
pseudo_time_series_test.to_pickle(data_folda+'skeleton_test_6month_master_google_retty_hotpepper_structured_tripadvisor.pkl')

del pseudo_time_series_test

/home/ec2-user/dglab-ai-scoring_insitestream/pg/./utils/rank.py:68: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df["n_rank_count"] = df["telephone_rank"].isin(["N"]).astype(int) + df["address_rank"].isin(["N"]).astype(int) + df["name_rank"].isin(["N"]).astype(int)
/home/ec2-user/dglab-ai-scoring_insitestream/pg/./utils/rank.py:73: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df["denominator"] = np.select(condition, choice, default=-1)
/home/ec2-user/dglab-ai-scoring_insitestream/pg/./utils/rank.py:78: Se

* V_HITOSARA から特徴量を追加

In [ ]:
# hitosara premium を追加

add_list=["restaurant_id","infos.premium","infos.onlineReservation"]

pseudo_time_series = pd.read_pickle(data_folda+'train_with_master_google_retty_hotpepper_structured_tripadvisor.pkl')
pseudo_time_series= reduce_mem_usage(pseudo_time_series)
pseudo_time_series = add_v_hitosara(pseudo_time_series, data_folda, add_list, master_name="V_HITOSARA",read_from_snowflake=True)
pseudo_time_series.to_pickle(data_folda+'train_with_master_structured_main_medias.pkl')

del pseudo_time_series

pseudo_time_series_test = pd.read_pickle(data_folda+'skeleton_test_3month_master_google_retty_hotpepper_structured_tripadvisor.pkl')
pseudo_time_series_test= reduce_mem_usage(pseudo_time_serie_test)
pseudo_time_series_test = add_v_hitosara(pseudo_time_series_test, data_folda, add_list, master_name="V_HITOSARA",read_from_snowflake=True) 
pseudo_time_series_test.to_pickle(data_folda+'test3_with_master_structured_main_medias.pkl')

pseudo_time_series_test = pd.read_pickle(data_folda+'skeleton_test_6month_master_google_retty_hotpepper_structured_tripadvisor.pkl')
pseudo_time_series_test= reduce_mem_usage(pseudo_time_serie_test)
pseudo_time_series_test = add_v_hitosara(pseudo_time_series_test, data_folda, add_list, master_name="V_HITOSARA",read_from_snowflake=True) 
pseudo_time_series_test.to_pickle(data_folda+'test6_with_master_structured_main_medias.pkl')

del pseudo_time_series_test

select * from KRAKEN_DG.KRAKEN_USER.V_HITOSARA


/home/ec2-user/dglab-ai-scoring_insitestream/pg/./utils/rank.py:68: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df["n_rank_count"] = df["telephone_rank"].isin(["N"]).astype(int) + df["address_rank"].isin(["N"]).astype(int) + df["name_rank"].isin(["N"]).astype(int)
/home/ec2-user/dglab-ai-scoring_insitestream/pg/./utils/rank.py:73: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df["denominator"] = np.select(condition, choice, default=-1)
/home/ec2-user/dglab-ai-scoring_insitestream/pg/./utils/rank.py:78: Se

: 

In [ ]:
read_from_snowflake=True
master_name="V_HITOSARA"

pd_sf = read_table(data_folda,master_name,columns="all",read_snowflake=read_from_snowflake,col="*")

# general rank を与える
ranker=rank.Ranker()
pd_sf=ranker.add_general_rank(pd_sf)
pd_sf=select_general_rank(pd_sf)
pd_sf.rename(columns={"general_rank":"general_rank_HITOSARA"},inplace=True)

#"infos.premium","infps.premium" の True, False を 1, 0 に変換
pd_sf["infos.premium"]=pd_sf["infos.premium"].replace('',np.nan).replace({"true":1,"false":0}).astype(int)
pd_sf["infos.onlineReservation"]=pd_sf["infos.onlineReservation"].replace('',np.nan).replace({"true":1,"false":0}).astype(int)



select * from KRAKEN_DG.KRAKEN_USER.V_HITOSARA


/home/ec2-user/dglab-ai-scoring_insitestream/pg/./utils/rank.py:68: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df["n_rank_count"] = df["telephone_rank"].isin(["N"]).astype(int) + df["address_rank"].isin(["N"]).astype(int) + df["name_rank"].isin(["N"]).astype(int)
/home/ec2-user/dglab-ai-scoring_insitestream/pg/./utils/rank.py:73: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df["denominator"] = np.select(condition, choice, default=-1)
/home/ec2-user/dglab-ai-scoring_insitestream/pg/./utils/rank.py:78: Se

In [ ]:
pseudo_time_series = pd.read_pickle(data_folda+'train_with_master_google_retty_hotpepper_structured_tripadvisor.pkl')



In [ ]:
pseudo_time_series.columns.values

array(['restaurant_id', 'original_id', 'closed_date', 'base_date',
       'target3', 'target6', 'eval_period', 'id', 'name',
       'prefecture_name', 'head_branch', 'opened_on', 'created_at',
       'genre_first_name', 'genre_second_name', 'genre_third_name',
       'northlatitude', 'eastlongitude', 'net_reservation_flg',
       'price_range_lunch_owner', 'price_range_dinner_owner',
       'price_range_lunch_user', 'price_range_dinner_user',
       'coupons_count', 'review_count', 'hozon_count', 'all_photo_count',
       'photo_food_count', 'photo_drink_count', 'photo_interior_count',
       'photo_exterior_count', 'real_opened_date', 'openning_days',
       'overview.aggregateRating', 'overview.reviewCount',
       'general_rank_GOOGLE', 'infos.isOfficial',
       'infos.familiar.users.genge', 'infos.familiar.users.count',
       'infos.familiar.stars', 'general_rank_RETTY',
       'infos.reserveEntireStore', 'infos.onlineReservation_x',
       'infos.access.transferTime1', 'infos.ac

In [ ]:
add_list=["restaurant_id","infos.premium","infos.onlineReservation"]
temp=pd_sf[add_list]

In [ ]:
pseudo_time_series= reduce_mem_usage(pseudo_time_series)


In [ ]:
temp.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 633110 entries, 401223 to 553329
Data columns (total 3 columns):
 #   Column                   Non-Null Count   Dtype 
---  ------                   --------------   ----- 
 0   restaurant_id            633110 non-null  object
 1   infos.premium            633110 non-null  int64 
 2   infos.onlineReservation  633110 non-null  int64 
dtypes: int64(2), object(1)
memory usage: 19.3+ MB


In [ ]:
pseudo_time_series.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 9628454 entries, 0 to 9628453
Columns: 122 entries, restaurant_id to infos.ratingDetails.Ambience
dtypes: bool(1), datetime64[ns](4), float32(46), float64(2), int64(12), object(56), timedelta64[ns](1)
memory usage: 7.1+ GB


In [ ]:
del pd_sf

In [ ]:
pseudo_time_series=pd.merge(pseudo_time_series,temp,on="restaurant_id",how="left")

: 